In [1]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State, ctx, Patch
import pandas as pd
import plotly
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
import copy
import os
import math
from io import StringIO
#import base64
#import io
#import numpy as np
#import time
#import scipy
#import requests
#from scipy import stats

In [2]:
libraries = {
    "Pandas" : pd,
    "Plotly" : plotly,
    "Dash" : dash
}

# On affiche les différentes versions des librairies utilisées
for lib_name, lib in libraries.items():
    print(f"{lib_name} Version : {lib.__version__}")

Pandas Version : 2.1.2
Plotly Version : 5.18.0
Dash Version : 2.14.1


In [3]:
import sys
print(sys.version)

3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]


In [6]:
# Get the current directory of the notebook
file_dir = os.getcwd()

# Go up one level to the parent directory (assuming the script is in the 'app' directory)
app_dir = os.path.dirname(file_dir)

# Define the path to the file in the /files directory
file_path = os.path.join(app_dir, 'files', 'donnees_nettoyees.csv')
#file_path='donnees_nettoyees.csv'
# Now you can use the file_path to access your file
with open(file_path, 'r') as file:
    data = pd.read_csv(file_path, sep = "\t")

In [7]:
data

,product_name,countries_en,nutriscore_score,pnns_groups_1,pnns_groups_2,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,fiber_100g,proteins_100g,salt_100g,Macronutrients
0,Tarte noix de coco,France,14.0,Composite foods,Pizza pies and quiches,381.0,1594.0,22.00,15.50,27.30,4.4,4.60,0.1000,54.0000
1,Compote de poire,France,-2.0,Fruits and vegetables,Fruits,157.0,657.0,0.00,0.00,36.00,3.6,0.60,0.0000,36.6000
2,BAguette bressan,France,-4.0,Cereals and potatoes,Bread,160.0,669.0,2.20,0.50,25.20,1.6,9.50,0.3580,37.2580
3,Cranberries,United States,6.0,Fruits and vegetables,Fruits,300.0,1255.0,0.00,0.00,83.33,10.0,0.00,0.0000,83.3300
4,Salade Cesar,Canada,14.0,Fruits and vegetables,Vegetables,290.0,1210.0,12.00,7.00,23.00,2.0,22.00,2.1600,59.1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152311,"Angelo & Marco, Mozzarella Cheese",United States,8.0,Milk and dairy products,Cheese,286.0,1197.0,21.43,14.29,0.00,0.0,21.43,0.1775,43.0375
152312,Mozzarella Ciliegine,United States,8.0,Milk and dairy products,Cheese,286.0,1197.0,21.43,14.29,0.00,0.0,21.43,0.1775,43.0375
152313,Ground beef burger with brisket,United States,11.0,Fish Meat Eggs,Meat,265.0,1109.0,20.59,8.24,0.00,0.0,17.65,0.1700,38.4100
152314,Pure Almond Butter,United States,-5.0,Fat and sauces,Fats,632.0,2644.0,60.71,7.14,21.43,3.6,14.29,0.0000,96.4300


In [8]:
def pnns_groups_options(df, country, pnns_groups_num, pnns1 = None):
    if pnns_groups_num == "pnns_groups_1":
        pnns_groups = df[pnns_groups_num].unique()
    elif pnns_groups_num == "pnns_groups_2":
        pnns_groups = df.loc[df.pnns_groups_1 == pnns1, pnns_groups_num].unique()

    pnns_groups = [
    {
        'label': f"{pnns} [{df[df.countries_en.str.contains(country)&(df[pnns_groups_num] == pnns)].shape[0]} products]",
        'value': pnns
    } 
    for pnns in pnns_groups
    ]

    return pnns_groups

In [9]:
# Function to generate a RangeSlider
def generate_slider(title, id, max_value):
    return html.Div([title,
                     dcc.RangeSlider(0, max_value, 1, value=[0, max_value],
                                    marks={0: "0", max_value: str(max_value)},
                                    id=id,
                                    tooltip={"placement": "bottom", "always_visible": True})
                     ], style={'textAlign': 'center', 'color': 'black', 'fontSize': 15})

In [10]:
# Function to generate a DropDown
def generate_dropdown(value, options, placeholder, multi, id):
    return dcc.Dropdown(
                value=value,
                options=options,
                style={'textAlign': 'left', 'color': 'black', 'fontSize': 15, 'width': '100%'},
                placeholder=placeholder,
                multi=multi,
                id=id
            )

In [11]:
def fig_graph_nutrients(df_slice, nutrients, nutrients_choice, ch_list_graph) :
    
    # To have Energy and the nutrients on the same graph
    figure_nutrients = make_subplots(specs=[[{"secondary_y": True}]])
    
    if nutrients_choice == []:
        nutrients_choice = None
    
    # No Figure
    if len(ch_list_graph) == 0 :#or country == None:
        return px.box()
    
    elif len(ch_list_graph) == 1:
        if "Distribution" in ch_list_graph :
            figure_nutrients1 = px.box(df_slice, y="energy_100g", hover_data=["product_name"]) 
            #figure_nutrients2 = px.box()
            #for nut in nutrients:
             #   figure_nutrients2.add_traces(px.box(mask, y=nut).data[0])
            figure_nutrients2 = px.box(df_slice, y=nutrients_choice, hover_data=["product_name"]) if nutrients_choice != None else px.box(df_slice, y=nutrients, hover_data=["product_name"])

        elif "Products" in ch_list_graph :
            figure_nutrients1 = px.strip(df_slice, y="energy_100g", hover_data=["product_name"]) 
            figure_nutrients2 = px.strip(df_slice, y=nutrients_choice, hover_data=["product_name"]) if nutrients_choice != None else px.strip(df_slice, y=nutrients, hover_data=["product_name"])                

    elif len(ch_list_graph) == 2:

        figure_nutrients1 = px.box(df_slice, y="energy_100g", hover_data=["product_name"])
        figure_nutrients1.add_trace(px.strip(df_slice, y="energy_100g", hover_data=["product_name"]).data[0])
        figure_nutrients1.update_traces(offsetgroup=0.5)

        figure_nutrients2 = px.box(df_slice, 
                                   y=nutrients_choice, 
                                   hover_data=["product_name"], 
                                   points = False) if nutrients_choice != None else px.box(df_slice, 
                                                      y=nutrients, 
                                                      hover_data=["product_name"], 
                                                      points = False
                                    )
        figure_nutrients2.add_trace(px.strip(df_slice, 
                                             y=nutrients_choice, 
                                             hover_data=["product_name"]
                                    ).data[0] if nutrients_choice != None else px.strip(df_slice, 
                                                y=nutrients, 
                                                hover_data=["product_name"]).data[0])
        figure_nutrients2.update_traces(offsetgroup=0.5)
            
    figure_nutrients1.update_traces(marker = dict(color = "red"))
    figure_nutrients2.update_traces(marker = dict(color = "green"))

    for i in range(len(figure_nutrients1.data)):
        figure_nutrients.add_trace(figure_nutrients1.data[i], secondary_y=False)
        figure_nutrients.add_trace(figure_nutrients2.data[i], secondary_y=True)

    # Update of figure layout
    figure_nutrients.update_layout(
        yaxis_title="g/100g",
        title=dict(text="Distribution of macronutrients of selected products",
                   font=dict(size=24, color="black"), x=0.5, xanchor='center'),
        font=dict(size=18, color="black"),
    )

    # Set y-axes titles
    figure_nutrients.update_yaxes(title_text="g/100g (energy)", secondary_y=False)
    figure_nutrients.update_yaxes(title_text="g/100g (nutrients)", secondary_y=True)
    
    return figure_nutrients

In [13]:
##### Initialize the app - incorporate css
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)

#server = app.server
app.title = 'Nutrition app'

versionning = "version: 0.0.2"

products_availability = "Referenced products: " + str(data.shape[0])

nutrients = ["fat_100g", "saturated-fat_100g", "carbohydrates_100g", "fiber_100g", "proteins_100g", "salt_100g", "Macronutrients"]

slider_trigger = ["slider_energy", "slider_fat", "slider_saturated", "slider_carbohydrates", "slider_fiber", "slider_proteins", "slider_salt", "slider_macronutrients"]

# Default setup
default_country, default_pnns1, default_pnns2 = "France", "Fruits and vegetables", "Soups"

# Options setup for dropdown of countries
c1 = [country.split(",") for country in data.countries_en.unique()]
c2 = [count for country in c1 for count in country]
unique_countries = sorted(list(set(c2)))

unique_countries = [
    {
        'label': f"{country} [{data[data.countries_en.str.contains(country)].shape[0]} products]",
        'value': country
    } 
    for country in unique_countries[1:]
]

app.layout = html.Div([
    
    # Informations
    html.Div([
        html.Div(className='row', children="Ruben HALIFA"),
    
        html.Div(className='row', children=versionning),
        
        html.Div(className='row', children=products_availability),
    ], style={'textAlign': 'left', 'color': 'black', 'fontSize': 12}),
    
    # Image of the dashboard
    html.Div(
        html.Img(src=dash.get_asset_url('pomme.jpeg'), 
             style={'width': '300px', 'height': '300px'}),
        style={'textAlign': 'center'}),
    
    # Title
    html.Div(className='row', children="Nutritious app",
             style={'textAlign': 'center', 'color': 'black', 'fontSize': 48}),
    
    # Horizontale line
    html.Hr(style={'border-top': '4px solid black'}), 
    
    # Dropdown for the countries
    html.Div([
        generate_dropdown(default_country, unique_countries, "Choose a country", False, 'dropdown_country')
    ], style={'margin': 'auto', 'width': '33%'}),
    
    # Dropdown for the pnns_groups_1
    html.Div([
        generate_dropdown(default_pnns1, [], "Choose a PNNS group 1", False, 'dropdown_pnns1')
    ], style={'display': 'inline-block', 'width': '50%'}),
    
    # Dropdown for the pnns_groups_2
    html.Div([
        generate_dropdown(default_pnns2, [], "Choose a PNNS group 2", False, 'dropdown_pnns2')
    ], style={'display': 'inline-block', 'width': '50%'}),
        
    # Dropdown for the macronutrient
    html.Div([
        generate_dropdown(None, nutrients, "Choose nutrients", True, 'dropdown_nutrients')
    ], style={'margin': 'auto'}),
    
    # Searchbar products
    html.Div([
        generate_dropdown(None, [], "Search a product", True, 'search_bar')
    ], style={'margin': 'auto'}),
    
    # Checklist type of graph
    html.Div([
        dcc.Checklist(
            value=["Distribution"],
            options=["Distribution", "Products"],
            style={'textAlign': 'center', 'color': 'black', 'fontSize': 15, 'width': '100%'},
            inline=True,
            id='check_list_graph')
    ], style={'margin': 'auto'}),
    
    html.Div([
        # Graph showing the distribution of the nutrients compare to the produce
        html.Div([
            dcc.Graph(id="graph_macronutrients", style={'height': '600px', 'width': '100%', 'float': 'left'}),
        ], style={'display': 'flex', 'flex-direction': 'row', 'width': '100%'}),

        # Sliders controling which products we show
        html.Div([
            # Button to reset sliders
            html.Div([
                html.Button(html.Strong("Reset"), id="reset_sliders_button", n_clicks=0, style={'color': 'black'})
                     ], style={'textAlign': 'center', 'color': 'black', 'fontSize': 15}),

                generate_slider("Energy kcal/100g", 'slider_energy', 3880),
                generate_slider("Fat g/100g", 'slider_fat', 100),
                generate_slider("Saturated_fat g/100g", 'slider_saturated', 100),
                generate_slider("Carbohydrates g/100g", 'slider_carbohydrates', 100),
                generate_slider("Fiber g/100g", 'slider_fiber', 100),
                generate_slider("Proteins g/100g", 'slider_proteins', 100),
                generate_slider("Salt g/100g", 'slider_salt', 100),
                generate_slider("Macronutrients g/100g", 'slider_macronutrients', 100)

            ], style={'width': '20%'}),
        ], style={'display': 'flex', 'flex-direction': 'row', 'width': '100%'}),
    
    # Table with data selection
    html.Div([
        html.Div(className='row', children="List of products by your search (max 20)",
             style={'textAlign': 'center', 'color': 'black', 'fontSize': 30}),
        html.Div( 
            dash_table.DataTable(
                data=None,
                page_size = 20,
                sort_action='native',
                sort_mode='multi',
                sort_by=[{'column_id':'nutriscore_score', 'direction':'asc'}], 
                id = "table_products"))
    ], style={'width': '100%'}),
    
    dcc.Store(id='initial_file', data=None),
    dcc.Store(id='intermed_file', data=None),
    dcc.Store(id='intermed_slide_file', data=None),
    dcc.Store(id='sliced_file', data=None),
])

@app.callback(
    Output('initial_file', 'data'),
    Output('intermed_file', 'data'),
    Output('intermed_slide_file', 'data'),
    Output('sliced_file', 'data'),
    
    Input('dropdown_country','value'),
    Input('dropdown_pnns1','value'),
    Input('dropdown_pnns2','value'),
    
    *[Input(f'{slide}', 'value') for slide in slider_trigger],
    
    State('initial_file', 'data'),
    State('intermed_file', 'data'),
    State('intermed_slide_file', 'data'),
    State('sliced_file', 'data'),
)

def data_slicing(country, pnns1, pnns2,
                 slide_energy, slide_fat, slide_sat_fat, slide_carbs, 
                 slide_fiber, slide_prot, slide_salt, slide_macro,
                 df_origin, df_intermediaire, df_inter_slide, df_slice):

    sliders = [slide_energy, slide_fat, slide_sat_fat, slide_carbs, slide_fiber, slide_prot, slide_salt, slide_macro]
    
    # Initial call
    if df_origin is None:
        df_origin = data.to_json(orient='split')
    
    # We do soething only if a country has been selected
    if country == None :
    
        return dash.no_update, None, None, None

    else :
        # Filtering based on country
        if ctx.triggered_id == "dropdown_country" or ctx.triggered_id is None:
            df_intermediaire = pd.read_json(StringIO(df_origin), orient='split')
            df_intermediaire = df_intermediaire[df_intermediaire.countries_en.str.contains(country)]

            df_inter_slide = copy.copy(df_intermediaire)
            
            # Verification of pnns conformity
            if pnns1 != None :
                df_inter_slide = df_inter_slide[df_inter_slide.pnns_groups_1 == pnns1]
            if pnns2 != None : 
                df_inter_slide = df_inter_slide[df_inter_slide.pnns_groups_2 == pnns2]

            df_slice = copy.copy(df_inter_slide)

            df_intermediaire = df_intermediaire.to_json(orient='split')
            df_inter_slide = df_inter_slide.to_json(orient='split')

            
        # Filtering based on pnns1
        if ctx.triggered_id in ["dropdown_pnns1"]:
            df_inter_slide = pd.read_json(StringIO(df_intermediaire), orient='split')
            # Verification of pnns conformity
            if pnns1 != None:
                df_inter_slide = df_inter_slide[df_inter_slide.pnns_groups_1 == pnns1]
                
            df_intermediaire, df_origin = dash.no_update, dash.no_update
            df_slice = copy.copy(df_inter_slide)
            df_inter_slide = df_inter_slide.to_json(orient='split')

        # Filtering based on pnns2
        if ctx.triggered_id in ["dropdown_pnns2"]:
            df_inter_slide = pd.read_json(StringIO(df_intermediaire), orient='split')
            # Verification of pnns conformity
            if (pnns1 != None) & (pnns2 != None):
                df_inter_slide = df_inter_slide[(df_inter_slide.pnns_groups_1 == pnns1) &
                                                (df_inter_slide.pnns_groups_2 == pnns2)]
                
            df_intermediaire, df_origin = dash.no_update, dash.no_update
            df_slice = copy.copy(df_inter_slide)
            df_inter_slide = df_inter_slide.to_json(orient='split')

        # Filtering based on slide
        if ctx.triggered_id in slider_trigger:

            df_slice = pd.read_json(StringIO(df_inter_slide), orient='split')
            
            df_intermediaire, df_origin, df_inter_slide = dash.no_update, dash.no_update, dash.no_update

        for nut, slide in zip(["energy_100g"] + nutrients, sliders):
            df_slice = df_slice[(df_slice[nut] >= slide[0]) & (df_slice[nut] <= slide[1])]

        df_slice = df_slice.to_json(orient='split')
        
    return df_origin, df_intermediaire, df_inter_slide, df_slice


@app.callback(
    Output('dropdown_country','value'),
    
    Input('dropdown_country','value'),
)

# We define the countries list
def choice_country(country):
    
    if country == []:
        country = None
        
    return country
    
@app.callback(
    Output('dropdown_pnns1','options'),
    Output('dropdown_pnns2','options'),
    Output('dropdown_pnns1','value'),
    Output('dropdown_pnns2','value'),
    
    Input('dropdown_pnns1','value'),
    Input('dropdown_pnns2','value'),
    Input('dropdown_country','value'),
    
    State('dropdown_pnns1','options'),
    State('dropdown_pnns2','options'),
)

# We define the pnns_groups
def choice_pnns_groups(pnns1, pnns2, country, pnns_groups_1, pnns_groups_2):
    
    # Modifying only if necessary
    if ctx.triggered_id == "dropdown_country":
        if country is None:
            
            pnns_groups_1, pnns_groups_2 = [], []
            pnns1, pnns2 = None, None
            
        else :
            pnns_groups_1 = pnns_groups_options(data, country, "pnns_groups_1")

            # Depending of pnns_groups_1 value
            if pnns1 != None:
                pnns_groups_2 = pnns_groups_options(data, country, "pnns_groups_2", pnns1)

                # Reset dropdown 
                if ctx.triggered_id == "dropdown_pnns1":
                    pnns2 = None
            else : 
                pnns_groups_2 = []
            
    # Reset dropdown pnns_groups_2 because they are not the same groups
    elif ctx.triggered_id == "dropdown_pnns1":
        
        pnns2 = None
        
        if pnns1 != None:
            pnns_groups_1 = dash.no_update
            pnns_groups_2 = pnns_groups_options(data, country, "pnns_groups_2", pnns1)
            
        else :
            pnns_groups_2 = []
        
    else:  
        pnns_groups_1 = dash.no_update
        pnns_groups_2 = dash.no_update
    
    # We verify the pnns1 and pnns2 values
    if pnns1 == []:
        pnns1 = None
    if pnns2 == []:
        pnns2 = None
        
    return pnns_groups_1, pnns_groups_2, pnns1, pnns2

    
@app.callback(
    *[Output(f'{slide}', 'min') for slide in slider_trigger],
    *[Output(f'{slide}', 'max') for slide in slider_trigger],
    *[Output(f'{slide}', 'marks') for slide in slider_trigger],
    *[Output(f'{slide}', 'value') for slide in slider_trigger],
    Input('intermed_slide_file', 'data'),
    Input('reset_sliders_button', 'n_clicks'),
    prevent_initial_call=True,
)
def update_sliders(df_inter_slide, n_clicks):
            
    # If we change the data
    if (df_inter_slide != None) & (ctx.triggered_id == "intermed_slide_file" or 
                                   ctx.triggered_id == "reset_sliders_button"):

        df_inter_slide = pd.read_json(StringIO(df_inter_slide), orient='split')
        output_values = []
        
        # Rounding down
        for nutrient in ["energy_100g"] + nutrients:
            nutrient_min = math.floor(df_inter_slide[f'{nutrient}'].min())
            output_values.extend([nutrient_min])
        # Rounding up
        for nutrient in ["energy_100g"] + nutrients:
            nutrient_max = math.ceil(df_inter_slide[f'{nutrient}'].max())
            output_values.extend([nutrient_max])
        for nutrient in ["energy_100g"] + nutrients:
            nutrient_min = math.floor(df_inter_slide[f'{nutrient}'].min())
            nutrient_max = math.ceil(df_inter_slide[f'{nutrient}'].max())
            nutrient_marks = {nutrient_min: str(nutrient_min), nutrient_max: str(nutrient_max)}
            output_values.extend([nutrient_marks])
        for nutrient in ["energy_100g"] + nutrients:
            nutrient_min = math.floor(df_inter_slide[f'{nutrient}'].min())
            nutrient_max = math.ceil(df_inter_slide[f'{nutrient}'].max())
            output_values.append([nutrient_min, nutrient_max])

        return tuple(output_values)

    return dash.no_update

@app.callback(
    Output('search_bar', 'options'),
    
    Input('intermed_slide_file', 'data'),
    prevent_initial_call=True,
)

def search_bar_options(df_inter_slide):
    if df_inter_slide is not None :
        df_inter_slide = pd.read_json(StringIO(df_inter_slide), orient='split')

        search_bar_options = df_inter_slide.product_name.sort_values().unique()

        return search_bar_options
    
    else :
        return dash.no_update

@app.callback(
    Output('table_products', 'data'),
    Output('table_products', 'style_data_conditional'),
    
    Input('table_products', "sort_by"),
    Input('sliced_file', 'data'),
    Input('search_bar', 'value'),
    
    State('intermed_slide_file', 'data')
)

def table_showing(sort_by, df_slice, search_bar_values, df_inter_slide):
        
    if df_slice != None :
        df_slice = pd.read_json(StringIO(df_slice), orient='split')

        # If we sort the table, we want the 20 best 
        if len(sort_by):
            df_slice.sort_values(
                [col['column_id'] for col in sort_by],
                ascending=[
                    col['direction'] == 'asc'
                    for col in sort_by
                ],
                inplace=True)
        
        
        # We show selected products
        if search_bar_values != None:
            df_inter_slide = pd.read_json(StringIO(df_inter_slide), orient='split')
            # We search for the selected products 
            df_inter_slide = df_inter_slide.loc[df_inter_slide.product_name.isin(search_bar_values)]
            
            # We don't show 2 times the same items
            df_slice = df_slice.loc[~df_slice.product_name.isin(search_bar_values)]
            
            # We concat the 20 best total (We retracte to 20 th number selected)
            if len(df_inter_slide) < 21 :
                concat_df = pd.concat([df_inter_slide, df_slice[:20 - len(df_inter_slide)]])
            else :
                concat_df = df_inter_slide

            concat_df.sort_values(
                [col['column_id'] for col in sort_by],
                ascending=[
                    col['direction'] == 'asc'
                    for col in sort_by
                ],
                inplace=True)
            
            # We gather the index 
            concat_df.reset_index(inplace=True, drop=True)
            concat_index = concat_df[concat_df['product_name'].isin(search_bar_values)].index.tolist()

            #concat_df['sorting_column'] = np.where(concat_df.product_name.isin(search_bar_values), 0, -1)
            #concat_df.sort_values(
            #    by=['sorting_column'] + [col['column_id'] for col in sort_by],
            #    ascending=[True] + [col['direction'] == 'asc' for col in sort_by],
            #    inplace=True
            #)
            #rows_to_style = df_slice[condition].index.tolist()
            style_data_conditional = []
            for row in concat_index:
                #color = "green" if row >= len(df_slice[:20 - len(df_inter_slide)]) else "tomato"
                #print(color, row, len(df_slice[:20 - len(df_inter_slide)])
                style_data_conditional.append({
                    'if': {'row_index': row},
                    'backgroundColor': "tomato" if row >= len(df_slice[:20 - len(df_inter_slide)]) else "green",
                    'color': 'white'
                })
        #df_slice.loc[df_slice['condition_to_exclude'], 'sorting_column'] = -1
            return concat_df.to_dict('records'), style_data_conditional
        
        else :
            return df_slice[:20].to_dict('records'), []
    
    # If no country selected, no data to show
    else : 
        return None, []

@app.callback(
    Output('graph_macronutrients', 'figure'),
    
    Input('dropdown_nutrients', 'value'),
    Input('check_list_graph', 'value'),
    Input('sliced_file', 'data'),
)

# We produce the main graphic depending of several input
def graph_macronutrients(nutrients_choice, ch_list_graph, df_slice):
    
    if df_slice != None :
        
        df_slice = pd.read_json(StringIO(df_slice), orient='split')
    
        return fig_graph_nutrients(df_slice, nutrients, nutrients_choice, ch_list_graph) 
    
    # If no country selected, no data to show
    else :
        return px.strip()
    
# Run the app
if __name__ == '__main__':
    app.run(debug=True)
    

2

In [ ]:
app.layout = html.Div([

In [331]:
data[:20].product_name.sort_values().unique()

array(['2 no chicken kievs', 'Apple Turnover', 'BAguette bressan',
       'Brioches roulées avec raisins', 'Chaussons tressés aux pommes',
       'Compote de poire', 'Cranberries', "Crème d'Artichaut Bio",
       'Dattes Mejdoul', 'Mon infusion', 'Muffin', 'Pain Burger Artisan',
       'Pastille Vichy', 'Pâte à Sucre', 'Quiche Lorraine', 'Root Beer',
       'Salade Cesar', 'Sliced Plain Bagel', 'Tarte noix de coco',
       'pesto'], dtype=object)

In [ ]:
df_slice = df_slice.loc[~df_slice.product_name.isin(search_bar_values)]

In [300]:
np.where(data[:20].product_name.isin(["Tarte noix de coco"]), 0, -1)

array([ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1])

In [289]:
mask = data.iloc[:10]
mask2 = data.iloc[15:17]
pd.concat([mask, mask2])

,product_name,countries_en,nutriscore_score,pnns_groups_1,pnns_groups_2,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,fiber_100g,proteins_100g,salt_100g,Macronutrients
0,Tarte noix de coco,France,14.0,Composite foods,Pizza pies and quiches,381.0,1594.0,22.00,15.500,27.30,4.40,4.60,0.10000,54.00000
1,Compote de poire,France,-2.0,Fruits and vegetables,Fruits,157.0,657.0,0.00,0.000,36.00,3.60,0.60,0.00000,36.60000
2,BAguette bressan,France,-4.0,Cereals and potatoes,Bread,160.0,669.0,2.20,0.500,25.20,1.60,9.50,0.35800,37.25800
3,Cranberries,United States,6.0,Fruits and vegetables,Fruits,300.0,1255.0,0.00,0.000,83.33,10.00,0.00,0.00000,83.33000
4,Salade Cesar,Canada,14.0,Fruits and vegetables,Vegetables,290.0,1210.0,12.00,7.000,23.00,2.00,22.00,2.16000,59.16000
5,Chaussons tressés aux pommes,Canada,9.0,Sugary snacks,Pastries,260.0,1090.0,10.70,2.000,38.70,2.00,3.33,0.64700,53.37700
6,Pain Burger Artisan,Canada,2.0,unknown,unknown,278.0,1160.0,1.11,0.333,53.30,2.22,10.00,1.52000,65.93000
7,pesto,"Belgium,France",22.0,Fat and sauces,Dressings and sauces,561.0,2318.0,53.20,9.600,11.70,1.30,8.20,2.70000,75.80000
8,Crème d'Artichaut Bio,"Belgium,France",7.0,Salty snacks,Salty and fatty products,268.0,1104.0,27.40,3.900,1.80,3.50,1.90,2.70000,33.80000
9,Root Beer,"France,United States",12.0,Beverages,Sweetened beverages,51.0,213.0,0.00,0.000,14.00,0.00,0.00,1.00000,15.00000


In [ ]:
@app.callback(
    *[Output(f'{slide}', 'min') for slide in slider_trigger],
    *[Output(f'{slide}', 'max') for slide in slider_trigger],
    *[Output(f'{slide}', 'marks') for slide in slider_trigger],
 
    Input('intermed_slide_file', 'data'),
)
# Modification of slider when changing selected products
def change_slider_pro(df_inter_slide):
    df_inter_slide = pd.read_json(StringIO(df_inter_slide), orient='split')

    output_values = []
    
    for nutrient in ["energy_100g"] + nutrients:
        nutrient_min = df_inter_slide[f'{nutrient}'].min()
        output_values.extend([nutrient_min])
    for nutrient in ["energy_100g"] + nutrients:
        nutrient_max = int(df_inter_slide[f'{nutrient}'].max())
        output_values.extend([nutrient_max])
    for nutrient in ["energy_100g"] + nutrients:
        nutrient_marks = {nutrient_min: str(nutrient_min), nutrient_max: str(nutrient_max)}
        output_values.extend([nutrient_marks])

    return tuple(output_values)




@app.callback(
    *[Output(f'{slide}', 'value') for slide in slider_trigger],
    
    Input('reset_sliders_button', 'n_clicks'),
    
    *[State(f'{slide}', 'min') for slide in slider_trigger],
    *[State(f'{slide}', 'max') for slide in slider_trigger],
    
    prevent_initial_call=True,
)

def reset_sliders_button(button_reset, *output_values):
    if ctx.triggered_id == "reset_sliders_button":
        return ([output_values[0], output_values[8]], [output_values[1], output_values[9]], [output_values[2],
                output_values[10]], [output_values[3], output_values[11]], [output_values[4], output_values[12]],
                [output_values[5], output_values[13]], [output_values[6], output_values[14]], 
                [output_values[7], output_values[15]])

In [6]:
    """mask = data.loc[(data.fat_100g >= slide_fat[0]) & (data.fat_100g <= slide_fat[1]) 
                   & (data["saturated-fat_100g"] >= slide_sat_fat[0]) & (data["saturated-fat_100g"] <= slide_sat_fat[1])
                   & (data.carbohydrates_100g >= slide_carbs[0]) & (data.carbohydrates_100g <= slide_carbs[1])
                   & (data.proteins_100g >= slide_prot[0]) & (data.proteins_100g <= slide_prot[1])
                   & (data.salt_100g >= slide_salt[0]) & (data.salt_100g <= slide_salt[1])
                   & (data.Macronutrients >= slide_macro[0]) & (data.Macronutrients <= slide_macro[1])
                   & (data.fiber_100g >= slide_macro[0]) & (data.fiber_100g <= slide_macro[1])]
    """
    
    """if ctx.triggered_id in ['slider_fat','slider_saturated','slider_carbohydrates',
                            'slider_fiber','slider_proteins','slider_salt','slider_macronutrients']:
                            """

"if ctx.triggered_id in ['slider_fat','slider_saturated','slider_carbohydrates',\n                        'slider_fiber','slider_proteins','slider_salt','slider_macronutrients']:\n                        "

In [ ]:
@app.callback(
    Output('slider_energy', 'min'),
    Output('slider_energy', 'max'),
    Output('slider_energy', 'marks'),
    Output('slider_fat', 'min'),
    Output('slider_fat', 'max'),
    Output('slider_saturated', 'min'),
    Output('slider_saturated', 'max'),
    Output('slider_carbohydrates', 'min'),
    Output('slider_carbohydrates', 'max'),
    Output('slider_fiber', 'min'),
    Output('slider_fiber', 'max'),
    Output('slider_proteins', 'min'),
    Output('slider_proteins', 'max'),
    Output('slider_salt', 'min'),
    Output('slider_salt', 'max'),
    Output('slider_macronutrients', 'min'),
    Output('slider_macronutrients', 'max'),
    Input('intermed_slide_file', 'data'),
)

# Modification of slider when changing selected products
def change_slider_pro(df_interm_slice):
    df_interm_slice = pd.read_json(StringIO(df_interm_slice), orient='split')
    
    energy = [df_interm_slice.energy_100g.min(), int(df_interm_slice.energy_100g.max())]
    energy_marks = {energy[0]: str(energy[0]), energy[1]: str(energy[1])}
    fat = [df_interm_slice.fat_100g.min(), int(df_interm_slice.fat_100g.max())]
    sat_fat = [df_interm_slice["saturated-fat_100g"].min(), int(df_interm_slice["saturated-fat_100g"].max())]
    carbs = [df_interm_slice.carbohydrates_100g.min(), int(df_interm_slice.carbohydrates_100g.max())]
    prots = [df_interm_slice.proteins_100g.min(), int(df_interm_slice.proteins_100g.max())]
    fibers = [df_interm_slice.fiber_100g.min(), int(df_interm_slice.fiber_100g.max())]
    salt = [df_interm_slice.salt_100g.min(), int(df_interm_slice.salt_100g.max())]
    macro = [df_interm_slice.Macronutrients.min(), int(df_interm_slice.Macronutrients.max())]

    return (energy[0], energy[1], energy_marks, fat[0], fat[1], sat_fat[0], sat_fat[1], carbs[0], carbs[1], prots[0], prots[1],
        fibers[0], fibers[1], salt[0], salt[1], macro[0], macro[1])